# 타이타닉 데이터셋을 불러와 `sibsp`, `parch` 컬럼을 더해 `family_size` 컬럼 생성

In [160]:
import pandas as pd
import seaborn as sns

data = sns.load_dataset('titanic')
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# 2. Feqture engineering이 적용된 데이터와 적용되지 않은 데이터를 각각 모델 학습에 사용해보고 결과 비교


In [161]:
data.info()
# age 결측치 -> 중간값?
# embarked 결측치 -> 2개 밖에 없으니 최빈값으로 ㄱㄱ
# sex, embarked 범주형 인코딩

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [162]:
# 중복값 제거
data = data.drop_duplicates()

In [163]:
# age 결측치를 중간값으로 대체
data['age'].fillna(data['age'].median(), inplace=True)

# embarked 결측치를 최빈값으로 대체
data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)

# sex, embarked 범주형 인코딩
data = pd.get_dummies(data, columns=['sex','embarked'], drop_first=True)

# 불필요한 컬럼 제거
data.drop(['alive', 'class','embark_town','who','adult_male','deck'], axis=1, inplace = True)


/var/folders/47/qvf6ns_n0xbb_s77tyz4nmbm0000gn/T/ipykernel_20544/1317285412.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['age'].fillna(data['age'].median(), inplace=True)
/var/folders/47/qvf6ns_n0xbb_s77tyz4nmbm0000gn/T/ipykernel_20544/1317285412.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

In [164]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 784 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   survived    784 non-null    int64  
 1   pclass      784 non-null    int64  
 2   age         784 non-null    float64
 3   sibsp       784 non-null    int64  
 4   parch       784 non-null    int64  
 5   fare        784 non-null    float64
 6   alone       784 non-null    bool   
 7   sex_male    784 non-null    bool   
 8   embarked_Q  784 non-null    bool   
 9   embarked_S  784 non-null    bool   
dtypes: bool(4), float64(2), int64(4)
memory usage: 45.9 KB


In [165]:
# 혼자 온 사람 생존률
solo = data[data['alone'] == 1]['survived'].mean() # 0.30

# 2~4명인 경우의 생존률
small_f = data[data['sibsp'] + data['parch'] >= 2]['survived'].mean() # 0.46

# 5명 이상인 경우의 생존률
big_f = data[data['sibsp'] + data['parch'] >= 5]['survived'].mean() # 0.14


# 'Family_size' 컬럼 추가
data['Family_Size'] = data['sibsp'] + data['parch'] + 1


In [166]:
# '1인당 요금(Fare_Per_Person)' 컬럼 추가
# 전체 요금을 가족 수로 나누기
data['Fare_Per_Person'] = data['fare'] / data['Family_Size']

In [167]:
# 나이대 구분 (Age_group) 컬럼 추가
data['Age_Group'] = pd.cut(data['age'],bins = [0,10,20,60,100],labels = ['child','Teen','Adult','Senior'])
data = pd.get_dummies(data, columns=['Age_Group'], drop_first=True)

In [168]:
# 불리언(True/False) 컬럼만 찾아서 정수(1/0)로 싹 바꿉니다
bool_cols = data.select_dtypes(include=['bool']).columns
data[bool_cols] = data[bool_cols].astype(int)

# 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 784 entries, 0 to 890
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   survived          784 non-null    int64  
 1   pclass            784 non-null    int64  
 2   age               784 non-null    float64
 3   sibsp             784 non-null    int64  
 4   parch             784 non-null    int64  
 5   fare              784 non-null    float64
 6   alone             784 non-null    int64  
 7   sex_male          784 non-null    int64  
 8   embarked_Q        784 non-null    int64  
 9   embarked_S        784 non-null    int64  
 10  Family_Size       784 non-null    int64  
 11  Fare_Per_Person   784 non-null    float64
 12  Age_Group_Teen    784 non-null    int64  
 13  Age_Group_Adult   784 non-null    int64  
 14  Age_Group_Senior  784 non-null    int64  
dtypes: float64(3), int64(12)
memory usage: 98.0 KB


## 원본 데이터의 경우

In [169]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 원본 데이터 준비
df_origin = sns.load_dataset('titanic')
df_origin.dropna(subset=['age', 'embarked'], inplace=True) # 결측치 제거
df_origin = pd.get_dummies(df_origin, columns=['sex', 'embarked'], drop_first=True)

# 피쳐와 타깃 분리
X_origin = df_origin[['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_male', 'embarked_Q', 'embarked_S']]
y_origin = df_origin['survived']

# 학습과 테스트를 위한 데이터셋 분리
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X_origin, y_origin, 
																									test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 준비
model = LogisticRegression(max_iter=1000)

# 모델 학습 및 예측
model.fit(X_train_o, y_train_o)
pred_o = model.predict(X_test_o)
# 예측 결과와 테스트 정답값을 비교해서 정확도 구하기
accuracy_o = accuracy_score(y_test_o, pred_o)
print("원본 데이터 정확도 : ", accuracy_o)

원본 데이터 정확도 :  0.7972027972027972


In [170]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 특성 공학으로 만든 데이터 준비
df_origin = data

# 피쳐와 타깃 분리
X_origin = df_origin[['pclass', 'age', 'sibsp', 'parch', 'fare', 'alone',
       'sex_male', 'embarked_Q', 'embarked_S', 'Family_Size',
       'Fare_Per_Person', 'Age_Group_Teen', 'Age_Group_Adult',
       'Age_Group_Senior']]
y_origin = df_origin['survived']

# 학습과 테스트를 위한 데이터셋 분리
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X_origin, y_origin, 
																									test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 준비
model = LogisticRegression(max_iter=1000)

# 모델 학습 및 예측
model.fit(X_train_e, y_train_e)
pred_e = model.predict(X_test_e)

# 예측 결과와 테스트 정답값을 비교해서 정확도 구하기
accuracy_e = accuracy_score(y_test_e, pred_e)
print("데이터 정확도 : ", accuracy_e)

데이터 정확도 :  0.7707006369426752


In [171]:
print(f"Feature Engineering 적용 전 정확도: {accuracy_o:.4f}")
print(f"Feature Engineering 적용 후 정확도: {accuracy_e:.4f}")
print(f"정확도 상승 폭 : {(accuracy_e - accuracy_o):.4f} %p")

Feature Engineering 적용 전 정확도: 0.7972
Feature Engineering 적용 후 정확도: 0.7707
정확도 상승 폭 : -0.0265 %p
